In [1]:
import pandas as pd
from pathlib import Path
from collections import defaultdict

In [17]:
INDEX = "SP500"
folder_path = Path(f"data{INDEX}")
files = [f.name for f in folder_path.iterdir() if f.is_file()]

In [18]:
posts_per_day = defaultdict(int)   #dict, inizializza a 0 key non esistenti

for file in folder_path.iterdir():
    ticker = file.stem.split("_")[0]
    try:
        df = pd.read_csv(file)
        if not df.empty:
            if "comments" in file.stem:
                df["date"] = pd.to_datetime(df["commentDate"], errors="coerce")
                time_delta = ( df["date"].max() - df["date"].min() ).days + 1
            else:
                df["date"] = pd.to_datetime(df["videoDate"], errors="coerce")
                time_delta = ( df["date"].max() - df["date"].min() ).days + 1 
            
            posts_per_day[f"{ticker}Tot"] += len(df)
            posts_per_day[ticker] += round( len(df) / time_delta , 1 )


    except Exception as e:
        print(f"Errore nel file {file.name}: {e}")

In [19]:
data = []
for ticker, count in posts_per_day.items():
    if "Tot" in ticker:
        tot = count
    else:
        data.append({ "Ticker": ticker, "TotElems": tot, "NumElemsPerDay": round( count, 1), "NumElemsPerWeek": round( count*7, 1) })


In [20]:
df = pd.DataFrame( data )
df.to_csv( f"num_elem_{INDEX}.csv", index=None )